In [59]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
import seaborn as sns
import json
import requests

In [60]:
path = "C:\\Users\\97254\\Desktop\\phyton\\exc2\\matala3\\"
filename = "dests.txt"
datafile =  path + filename
data = pd.read_csv(datafile, header=None)
cities = []
for i in data[0] :
    cities.append(i)

In [61]:
address="Ariel University, Ariel, Israel"
api_key = 'AIzaSy____1SQXUOA'

In [62]:
def get_lat_lng(address_string):
    import requests
    #python code goes here
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address_string,api_key)
    response = requests.get(url)
    response_data = response.json()
    return  response_data
#get_lat_lng(address)


In [63]:
def get_distance_duration(destination):
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json'
    params = {
        'origins': 'Tel Aviv, Israel',
        'destinations': destination,
        'key': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    distance_km = data['rows'][0]['elements'][0]['distance']['value'] / 1000
    duration_min = data['rows'][0]['elements'][0]['duration']['value'] / 60
    return {'distance_km': distance_km, 'duration_min': duration_min}

In [64]:
# Define a function to retrieve longitude and latitude from Google Geocoding API
def get_location(destination):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': destination,
        'key': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    location = data['results'][0]['geometry']['location']
    return {'longitude': location['lng'], 'latitude': location['lat']}

In [65]:
results = []
for destination in cities:
    distance_duration = get_distance_duration(destination)
    location = get_location(destination)
    result = {'target': destination}
    result.update(distance_duration)
    result.update(location)
    results.append(result)

# Create a dataframe from the results
df = pd.DataFrame(results, columns=['target', 'distance_km', 'duration_min', 'longitude', 'latitude'])

In [66]:
df['duration'] = df['duration_min'].apply(lambda x: f"{int(x) // 60}h {int(x) % 60}m")
df = df.drop(columns=['duration_min'])
df= df[['target','distance_km','duration','longitude','latitude']]
df

,target,distance_km,duration,longitude,latitude
0,Istanbul,1815.227,21h 2m,28.978359,41.008238
1,Amsterdam,4527.132,48h 16m,4.904139,52.367573
2,Valletta,3792.883,50h 50m,14.514100,35.899237
3,Basel,4092.805,44h 2m,7.588576,47.559599
4,Doha,2164.477,22h 38m,51.531040,25.285447


In [67]:
#farthest_cities = df.sort_values(by='distance_km', ascending=False).head(3)['target']
#print(farthest_cities.to_string())

In [68]:
farthest_cities = df.nlargest(3, 'distance_km')['target']
print('The 3 cities furthest from Tel Aviv are:')
print(farthest_cities.to_string())

The 3 cities furthest from Tel Aviv are:
1    Amsterdam
3        Basel
2     Valletta
